In [1]:
# Check GPU Specs
!nvidia-smi

Tue Jun  9 13:12:30 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
# Get hashes to crack
# Make a directory called 'dothashcat' in your Google Drive before this
# with the 'hashes' subdirectory for storing hashes
from google.colab import drive

# This will ask you to go to a link and get an authorization code
# to let the .hashcat folder get synced to your Google Drive
drive.mount('/content/drive')

# Make a symbolic link between Google Drive
# and the local .hashcat directory
!ls /content/drive/My\ Drive/dothashcat
!ln -s /content/drive/My\ Drive/dothashcat /root/.hashcat

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
dothashcat	   hashcat.potfile  kernels    sessions
hashcat.dictstat2  hashes	    plaintext


In [3]:
# Clone, build and install Hashcat
!apt install cmake build-essential -y
!apt install checkinstall git -y
!git clone https://github.com/hashcat/hashcat.git
!cd hashcat && git submodule update --init && make && make install

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
cmake is already the newest version (3.10.2-1ubuntu2.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 32 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.17.1-1ubuntu0.7).
The following additional packages will be installed:
  file libmagic-mgc libmagic1
Suggested packages:
  gettext
The following NEW packages will be installed:
  checkinstall file libmagic-mgc libmagic1
0 upgraded, 4 newly installed, 0 to remove and 32 not upgraded.
Need to get 372 kB of archives.
After this operation, 5,735 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic-mgc amd64 1:5.32-2ubuntu0.4 [184 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libmagic1 amd64 1:5

In [4]:
# Check the Hashcat Version to make sure it's installed correctly
!hashcat --version

v5.1.0-1846-gf92df252


In [5]:
# WPA2 EAPOL Benchmark (change this to check the speed of whichever hash you're cracking)
!hashcat -w 4 -m 2500 --benchmark

hashcat (v5.1.0-1846-gf92df252) starting in benchmark mode...

* Device #1: This hardware has outdated CUDA compute capability (3.7).
             For modern OpenCL performance, upgrade to hardware that supports
             CUDA compute capability version 5.0 (Maxwell) or higher.
* Device #2: This hardware has outdated CUDA compute capability (3.7).
             For modern OpenCL performance, upgrade to hardware that supports
             CUDA compute capability version 5.0 (Maxwell) or higher.
nvmlDeviceGetFanSpeed(): Not Supported

CUDA API (CUDA 10.1)
* Device #1: Tesla K80, 11373/11441 MB, 13MCU

OpenCL API (OpenCL 1.2 CUDA 10.1.152) - Platform #1 [NVIDIA Corporation]
* Device #2: Tesla K80, skipped

Benchmark relevant options:
* --workload-profile=4

Hashmode: 2500 - WPA-EAPOL-PBKDF2 (Iterations: 4095)

Speed.#1.........:    88331 H/s (300.57ms) @ Accel:16 Loops:512 Thr:1024 Vec:1

Started: Tue Jun  9 13:15:14 2020
Stopped: Tue Jun  9 13:15:36 2020


In [6]:
# Get wordlists
!mkdir /root/wordlists && cd /root/wordlists

# Clone the SecLists repository
!git clone https://github.com/danielmiessler/SecLists /root/wordlists/SecLists

# Append 'hashcat' for demonstration purposes
!echo "hashcat!" >> /root/wordlists/SecLists/Passwords/Leaked-Databases/rockyou-75.txt

!ls /root/wordlists/SecLists

Cloning into '/root/wordlists/SecLists'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 9074 (delta 3), reused 0 (delta 0), pack-reused 9063
Receiving objects: 100% (9074/9074), 692.14 MiB | 21.24 MiB/s, done.
Resolving deltas: 100% (4673/4673), done.
Checking out files: 100% (5303/5303), done.
CONTRIBUTING.md  Fuzzing  Miscellaneous     Payloads   Web-Shells
CONTRIBUTORS.md  IOCs	  Passwords	    README.md
Discovery	 LICENSE  Pattern-Matching  Usernames


In [7]:
# Check that your hashes are accessible to Colab
!ls /root/.hashcat/hashes

test.hccapx


In [11]:
# Start pure brute force cracking with a session name
# !hashcat -m 2500 -a3 --session $SESSION /root/.hashcat/hashes/test.hccapx
# !hashcat -m 2500 /root/.hashcat/hashes/test.hccapx -a 3 -w 4 --session test-session

# Restore the session
# !hashcat --session $SESSION --restore

# Crack with a wordlist
!hashcat -m 2500 /root/.hashcat/hashes/test.hccapx /root/wordlists/SecLists/Passwords/Leaked-Databases/rockyou-75.txt

hashcat (v5.1.0-1846-gf92df252) starting...

* Device #1: This hardware has outdated CUDA compute capability (3.7).
             For modern OpenCL performance, upgrade to hardware that supports
             CUDA compute capability version 5.0 (Maxwell) or higher.
* Device #2: This hardware has outdated CUDA compute capability (3.7).
             For modern OpenCL performance, upgrade to hardware that supports
             CUDA compute capability version 5.0 (Maxwell) or higher.
nvmlDeviceGetFanSpeed(): Not Supported

CUDA API (CUDA 10.1)
* Device #1: Tesla K80, 11373/11441 MB, 13MCU

OpenCL API (OpenCL 1.2 CUDA 10.1.152) - Platform #1 [NVIDIA Corporation]
* Device #2: Tesla K80, skipped

Minimum password length supported by kernel: 8
Maximum password length supported by kernel: 63

Hashes: 1 digests; 1 unique digests, 1 unique salts
Bitmaps: 16 bits, 65536 entries, 0x0000ffff mask, 262144 bytes, 5/13 rotates
Rules: 1

Applicable optimizers:
* Zero-Byte
* Single-Hash
* Single-Salt
* Slo